# Knihovna Scikit-learn, přehled užitečných funkcí

V předešlé hodině jsme se vyhýbali programování, jak jen se to dalo. Teď už si ale chceš také sama vše vyzkoušet.
Abys mohla úlohu rozmyšlenou v domácím úkolu naprogramovat, projdeme si nejdůležitější funkce, které budeš potřebovat. 

Především budeme používat knihovnu [Sciki-learn](https://scikit-learn.org) a samozrejmě také pandas. 
Potřebné věci projdeme na příkladu. 

In [1]:
import pandas as pd

## Načtení a příprava dat 

Na začátku vždy bude potřeba připravit data. Čištění dat a použití knihovny pandas už bys měla ovládat, 
zameříme se jen na věci, které jsou specifické pro strojové učení.

Načíst data tedy umíš.

In [2]:
df_platy = pd.read_csv("static/salaries.csv", index_col=0)
df_platy.sample(10)

,rank,discipline,yrs.since.phd,yrs.service,sex,salary
33,Prof,B,13,9,Male,117256
113,AsstProf,A,3,1,Male,72500
6,AssocProf,B,6,6,Male,97000
196,AssocProf,B,9,7,Male,113600
14,AsstProf,B,2,0,Male,78000
158,AsstProf,B,1,0,Male,88000
189,AssocProf,B,28,28,Male,106300
108,AssocProf,A,10,8,Male,82600
190,Prof,B,25,19,Male,153750
157,AssocProf,B,12,18,Male,113341


Pro predikci použijeme jako příznaky `rank`, `discipline`, `yrs.since.phd`, `yrs.service` a `sex`, 
predikovat budeme hodnotu `salary`.  

Pro učení potřebujeme všechny hondoty převést na čísla (`float`). Pokud by data obsahovala chybějící
hodnoty, nejjednodušší řešení je takové řádky zahodit. (Bonus: pokud bys měla data s větším množstvím
chybějících hodnot, podívej se na možnosti [sklearn.impute](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.impute))

Důležité je vypořádat se s kategorickými hodnotami. Sloupce obsahující hodnoty typu Boolean nebo dvě hodnoty (např. muž/žena), lze snadno převést na hodnoty $[0,1]$. 

In [3]:
df_platy = df_platy.replace({"Male": 0, "Female": 1})
df_platy.sample(10)

,rank,discipline,yrs.since.phd,yrs.service,sex,salary
131,AssocProf,A,11,9,0,83001
149,Prof,B,36,26,1,144651
70,Prof,B,28,36,0,91412
141,AssocProf,A,14,8,0,100102
56,AssocProf,B,14,5,0,83900
98,Prof,B,13,7,0,129676
133,AssocProf,A,10,8,1,77500
142,AssocProf,A,15,10,0,81500
1,Prof,B,19,18,0,139750
53,AsstProf,B,11,3,1,74692


Pro kategorické proměnné s více možnostvi použijeme tzv. *one-hot-ecoding*. 

Př. sloupec `rank` obsahuje hodnoty `Prof`, `AsstProf` a `AssocProf`. K zakódování pomocí one-hot-encoding potřebujeme tři sloupce: 

Původní hodnota | Kód 
--- | --- 
Prof      | 1 0 0
AsstProf  | 0 1 0 
AssocProf | 0 0 1  


Knihovna Scikitlearn nabízí [sklearn.preprocessing.OneHotEncoder](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html#sklearn.preprocessing.OneHotEncoder), při práci s pandas však můžeme použít rovnou metodu [get_dummies](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.get_dummies.html). (Pozn. *dummies* proto, že nám přibudou pomocné proměnné (sloupce), které se označují jako *dummy variables*.)

In [4]:
df_platy = pd.get_dummies(df_platy)
df_platy

,yrs.since.phd,yrs.service,sex,salary,rank_AssocProf,rank_AsstProf,rank_Prof,discipline_A,discipline_B
1,19,18,0,139750,0,0,1,0,1
2,20,16,0,173200,0,0,1,0,1
3,4,3,0,79750,0,1,0,0,1
4,45,39,0,115000,0,0,1,0,1
5,40,41,0,141500,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...
194,19,19,0,86250,1,0,0,0,1
195,48,53,0,90000,1,0,0,0,1
196,9,7,0,113600,1,0,0,0,1
197,4,4,0,92700,0,1,0,0,1


Poslední krok předzpracování bývá přeškálování hodnot. Není to vždy nutné, ale některým modelům to může pomoci.
Využijeme [StandardScaler](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html#sklearn.preprocessing.StandardScaler).
(Pozn.: vytvoříme zvlášť objekt na transformaci odezvy, abychom ho později mohli použít samostatně na konverzi
predikovaných hodnot). 

In [5]:
from sklearn.preprocessing import StandardScaler 

priznaky_ke_konverzi = ["yrs.since.phd", "yrs.service"]
odezva = ["salary"]

transformace = StandardScaler()
df_platy[priznaky_ke_konverzi] = transformace.fit_transform(df_platy[priznaky_ke_konverzi])

transformace_odezva = StandardScaler()
df_platy[odezva] = transformace_odezva.fit_transform(df_platy[odezva])

df_platy.sample(10)


,yrs.since.phd,yrs.service,sex,salary,rank_AssocProf,rank_AsstProf,rank_Prof,discipline_A,discipline_B
180,-1.319785,-1.024448,1,-0.669128,0,1,0,0,1
36,-1.156717,-1.276633,1,-1.194624,0,1,0,0,1
45,-0.015236,0.320538,0,-0.585609,0,0,1,0,1
156,0.147832,0.488661,0,0.275749,0,0,1,0,1
118,1.615450,1.749586,0,0.224741,0,0,1,1,0
74,1.289313,1.581463,0,-0.655430,0,0,1,0,1
26,0.147832,-0.604140,0,-0.168365,0,0,1,1,0
150,-1.238251,-1.024448,0,-0.561261,0,1,0,0,1
138,-0.178305,-0.099770,0,-0.190296,0,0,1,1,0
53,-0.667511,-1.024448,1,-1.275480,0,1,0,0,1


## Vytvoření trénovací a testovací množiny

V teorii strojového učení se vstupy modelu (příznaky, vsupní proměnné) typicky označují písmenem `X` a výstupy písmenem `y`. Řada programátorů toto používá i k označování proměnných v kódu. 
`X` představuje *matici* (neboli tabulku), kde každý řádek odpovídá jednomu datovému vzorku a každý sloupec jednomu příznaku (vstupní proměnné). `y` je vektor, neboli jeden sloupec s odezvou. 

Na vyzobnutí odezvy se výborně hodí metoda [pop](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.pop.html).

In [6]:
y = df_platy.pop("salary")
X = df_platy 

print(X.columns)
print(y.name)

Index(['yrs.since.phd', 'yrs.service', 'sex', 'rank_AssocProf',
       'rank_AsstProf', 'rank_Prof', 'discipline_A', 'discipline_B'],
      dtype='object')
salary


Zbývá data rozdělit na trénovací a testovací. K tomu slouží metoda [train_test_split](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html?highlight=train%20test%20split#sklearn.model_selection.train_test_split). 
Data nám rozdělí náhodně na trénovací a testovací sadu. Velikost testovací množiny můžeme specifikovat parametrem `test_size`, jeho defaultní hodnota je `0.25`, t. j. 25%.

In [7]:
from sklearn.model_selection import train_test_split 

X_train, X_test, y_train, y_test = train_test_split(X, y)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

## Modely 

Můžeme přejít k samotnému učení. Vybereme si model. Přehled modelů najdeš v sekci [Supervised learnig](https://scikit-learn.org/stable/supervised_learning.html#supervised-learning).
                                                                                                       
                                        
Na regresi můžeš použít:
  - TODO
   -
 
Na klasifikační úlohy (ke kterým se dostaneme v této hodině) využiješ: 
  - TODO
  -

Vytvoříme instanci vybraného modelu (jde nám teď o syntaxi, vezmeme nejjednodušší lineární regresi):

In [8]:
from sklearn.linear_model import LinearRegression

model = LinearRegression()

## Trénování

Model natrénujeme na trénovací množině:

In [9]:
model.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

## Predikce 

Natrénovaný model typicky chceme použít k ohodnocení nějakých nových datových vzorků, k tomu máme metodu `predict`. Zavolejme ji jak na trénovací, tak na testovací data.

In [10]:
train_predikce = model.predict(X_train)
test_predikce = model.predict(X_test)

Pozor, k učení jsme použili transformované hodnoty odezvy. Zajímají-li nás skutečné hodnoty platů, 
musíme i predikované hodnoty transformovat zpět.

In [11]:
predikce_platu = pd.Series(transformace_odezva.inverse_transform(test_predikce))
predikce_platu.head()

0     99634.807337
1    136346.076453
2    100747.633295
3     84904.657267
4    133836.823086
dtype: float64

## Evaluace modelu

Můžeme využít funkci `score`, která nám vrátí hodnotu $R^2$ metriky:   

In [12]:
print("R2 na trénovací množině: ", model.score(X_train, y_train))
print("R2 na testovací množině: ", model.score(X_test, y_test))

R2 na trénovací množině:  0.527298223376352
R2 na testovací množině:  0.5006205497534375


Funkce pro všechny možné metriky najdeš v [sklearn.metrics](https://scikit-learn.org/stable/modules/classes.html?highlight=sklearn%20metrics#module-sklearn.metrics).
                                                            (nyní nás zajímají [regresní metriky](https://scikit-learn.org/stable/modules/classes.html?highlight=sklearn%20metrics#module-sklearn.metrics))          

In [13]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

pd.DataFrame(
    {
        "MAE": [mean_absolute_error(y_train, train_predikce), mean_absolute_error(y_test, test_predikce)],
        "MSE": [mean_squared_error(y_train, train_predikce), mean_absolute_error(y_test, test_predikce)],
        "R2": [r2_score(y_train, train_predikce), r2_score(y_test, test_predikce)]
    },
    index = ["train", "test"]
)

,MAE,MSE,R2
train,0.526284,0.525795,0.527298
test,0.434131,0.434131,0.500621


## Uložení modelu 

Někdy si potřebujeme naučený model uchovat na další použití. Model lze uložit do souboru a zase načíst pomocí `pickle`.
Kujme pikle:

In [14]:
import pickle 

with open("model", "wb") as soubor:
    pickle.dump(model, open("model.pickle", "wb"))


with open("model.pickle", "rb") as soubor:
    staronovy_model = pickle.load(soubor)

staronovy_model.score(X_test, y_test)

0.5006205497534375

Pozn.: Bohužel mám poměrně bohaté špatné zkušenosti z načítáním modelů uložených před delším časem
(bývá problém načíst model uložený ve starší verzi Scikit-learn ve verzi novější). 